## Filter and align waveforms

In [ ]:
import warnings

import pandas as pd
import numpy as np

from utils import Dataset

warnings.filterwarnings("ignore")

dataset = Dataset("../data/catalog.pkl")
print(f"{len(dataset.catalog)} total events in catalog")

Filter out waveforms with high energy before the trigger

In [ ]:
threshold = 15
n = dataset.catalog.count()["expname"]
dataset.catalog = dataset.catalog[dataset.catalog["init_energy"]<threshold]
n1 = dataset.catalog.count()["expname"]
print(f"{n-n1} events excluded with energy thresholding")

In [ ]:
# window around the trigger for training models
w_min = 350
w_max = 500

In [ ]:
# remove low frequency noisy waveforms

def zero_crossings(a):
    n_ = np.where(np.diff(np.signbit(a)))[0]
    return len(n_)

dataset.catalog["ZeroCrossing_arrival"] = dataset.catalog["normedwaves"].apply(lambda x: zero_crossings(x[w_min:w_max]))

zerocross_thresh = 5
dataset.catalog = dataset.catalog[dataset.catalog["ZeroCrossing_arrival"]>zerocross_thresh]
n2 = dataset.catalog.count()["expname"]
print(f"{n1-n2} events excluded with zero crossing processing")
print(f"Total kept {n2}")

In [ ]:
pd.DataFrame(dataset.catalog.groupby("expname").count()["waveforms"])

In [ ]:
# realign waveforms to first peak after trigger

offsets = dataset.catalog.normedwaves.apply(lambda x: np.argmax(x[350+45:350+60]))

alignedwaves = []
for i, offset in enumerate(offsets):
    alignedwaves.append(np.roll(dataset.catalog.normedwaves.iloc[i], -offset))

dataset.catalog["alignedwaves"] = alignedwaves

In [ ]:
# save 
dataset.catalog.to_pickle("../data/final_filtered_catalog_for_model_training_and_eval.pkl")